# XGBoost regression (single-node)

<table>
    <tr>
        <td>
            <img src="../_img/xgboost.png" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="450">
        </td>
    </tr>
</table>

This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll load data from Snowflake into a `pandas` data frame and use XGBoost to answer this question

> based on characteristics that can be known at the beginning of a trip, what tip will this trip earn (as a % of the total fare)?

**NOTE:** This notebook has some cells that can take 3-10 minutes to run. Consider opening [this Dask + XGBoost notebook](./xgboost-dask.ipynb) and running that while you're waiting for cells in this notebook to complete.

<hr>

## Connect to Snowflake

This example uses data stored in a Snowflake data warehouse that is managed by the team at Saturn Cloud. We've set up a read-only user for use in these examples. If you would like to access data stored in your own Snowflake account, you should set up [Credentials](https://www.saturncloud.io/docs/concepts/credentials/) for your account, user, and password then set the other connection information accordingly. For more details on Snowflake connection information, see ["Connecting to Snowflake"](https://docs.snowflake.com/en/user-guide/python-connector-example.html#connecting-to-snowflake) in the `snowflake-connector-python` docs.

Note that in order to update environment variables your Jupyter server will need to be stopped.

In [ ]:
import os
import snowflake.connector
import pandas as pd

conn_info = {
    "account": os.environ["EXAMPLE_SNOWFLAKE_ACCOUNT"],
    "user": os.environ["EXAMPLE_SNOWFLAKE_USER"],
    "password": os.environ["EXAMPLE_SNOWFLAKE_PASSWORD"],
    "database": os.environ["TAXI_DATABASE"],
}
conn = snowflake.connector.connect(**conn_info)

<hr>

## Load data

This example is designed to run quickly with small resources. So let's just load a single month of taxi data for training.

This example uses Snowflake to handle the hard work of creating new features, then creates a `pandas` data frame with the result.

In [ ]:
query = """
SELECT * FROM (
    SELECT
        pickup_taxizone_id,
        dropoff_taxizone_id,
        passenger_count,
        DIV0(tip_amount, fare_amount) AS tip_fraction,
        DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
        WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
        HOUR(pickup_datetime) AS pickup_hour,
        (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
        MINUTE(pickup_datetime) AS pickup_minute
    FROM taxi_yellow
    WHERE
        DATE_TRUNC('MONTH', pickup_datetime) = '{day}'
) SAMPLE (30)
"""
taxi = pd.read_sql(query.format(day="2019-01-01"), conn)
taxi.columns = taxi.columns.str.lower()

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum() / 1e6} MB")

In [ ]:
numeric_feat = [
    "pickup_weekday",
    "pickup_weekofyear",
    "pickup_hour",
    "pickup_week_hour",
    "pickup_minute",
    "passenger_count",
]
categorical_feat = [
    "pickup_taxizone_id",
    "dropoff_taxizone_id",
]
features = numeric_feat + categorical_feat
y_col = "tip_fraction"

taxi_train = taxi[features + [y_col]].astype(float).fillna(-1)

In [ ]:
taxi_train.head()

<hr>

## Train a Model

Now that the data have been prepped, it's time to build a model!

For this task, we'll use the `XGBRegressor` from `xgboost`. If you've never used XGBoost or need a refresher, see [the XGBoost Python docs](https://xgboost.readthedocs.io/en/latest/python/index.html).

The code below initializes an `XGBRegressor` with the following parameter values.

* `objective = "reg:squarederror"`: solve a regression problem, and use mean squared error as the loss function
* `tree_method = "hist"`: use the "histogram" method of building trees. This method sacrifices a small amount of training accuracy for much faster training time.
* `learning_rate = 0.1`: controls how much each new tree contributes to the overall model. 0.1 is an arbitrary value.
* `max_depth = 5`: stop growing a tree once it contains a leaf node that is 5 levels below the root
* `n_estimators = 50`: create a model with at most 50 trees
* `n_jobs = multiprocessing.cpu_count()`: use all available cores on this machine to parallelize training
* `verbosity = 1`: print INFO-level logs and above

All other parameters use the defaults from [`XGBRegressor`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor).

<details><summary>(click here to learn why data scientists do this)</summary>

**Setting max_depth**
    
Tree-based models split the training data into smaller and smaller groups, to try to group together records with similar values of the target. A tree can be thought of as a collection of rules like `pickup_hour greater than 11` and `pickup_minute less than 31.0`. As you add more rules, those groups (called "leaf nodes") get smaller. In an extreme example, a model could create a tree with enough rules to place each record in the training data into its own group. That would probably take a lot of rules, and would be referred to as a "deep" tree.
    
Deep trees are problematic because their descriptions of the world are too specific to be useful on new data. Imagine training a classification model to predict whether or not visitors to a theme park will ride a particular rollercoaster. You could measure the time down to the millisecond that every guest's ticket is scanned at the entrance, and a model might learn a rule like *"if the guest has been to the park before and if the guest is older than 40 and younger than 41, and if the guest is staying at Hotel A and if the guest enters the park after 1:00:17.456 and if the guest enters the park earlier than 1:00:17.995, they will ride the rollercoaster"*. This is very very unlikely to ever match any future visitors, and if it does it's unlikely that this prediction will be very good unless you have some reason to believe that a visitor arriving at 1:00:18 instead of 1:00:17 really changes the probability that they'll ride that rollercoaster.
    
To prevent this situation (called "overfitting"), most tree-based machine learning algorithms accept parameters that control how deep the trees can get. `max_depth` is common, and says "don't create a rule more complex than this". In the example above, that rule has a depth of 7.
    
1. visiting the park
2. has been to the park before?
3. older than 40?
4. younger than 41?
5. staying at Hotel A?
6. entered the park after 1:00:17.456?
7. entered the park before 1:00:17.995?
    
Setting `max_depth = 5` would have prevented those weirdly-specific timing rules from ever being generated.
    
Choosing good values for this parameter is part art, part science, and is outside the scope of this tutorial.
    
</details>

In [ ]:
import xgboost
import multiprocessing

xgb_reg = xgboost.XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",
    learning_rate=0.1,
    max_depth=5,
    n_estimators=50,
    n_jobs=multiprocessing.cpu_count(),
    verbosity=1,
)

With the regressor created, fit it to some data! The code below uses `%%time` to print out a timing, so you can see how long it takes to train. This can be used to compare single-node, CPU `xgboost` to methods explored in other notebooks, or to test how changing some parameters changes the runtime for training.

**NOTE:** This will take a few minutes to run. Consider opening [this Dask + XGBoost notebook](./xgboost-dask.ipynb) and running that while you're waiting for this model to train.

In [ ]:
%%time
_ = xgb_reg.fit(taxi_train[features], y=taxi_train[y_col])

<hr>

## Save model

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/xgboost.pkl", "wb") as f:
    cloudpickle.dump(xgb_reg, f)

<hr>

## Calculate metrics on test set

Machine learning training tries to create a model which can produce useful results on new data that it didn't see during training. To test how well we've accomplished that in this example, read in another month of taxi data.

In [ ]:
taxi_test = pd.read_sql(query.format(day="2019-02-01"), conn)
taxi_test.columns = taxi_test.columns.str.lower()
taxi_test = taxi_test.astype(float).fillna(-1)

`scikit-learn` comes with many functions for calculating metrics that describe how well a model's predictions match the actual values. For a complete list, see ["Metrics and scoring"](https://scikit-learn.org/stable/modules/model_evaluation.html) in the `sciki-learn` docs.

This tutorial uses the `mean_squared_error` to evaluate the model. This metric penalizes large errors more than small errors.

In [ ]:
from sklearn.metrics import mean_squared_error

preds = xgb_reg.predict(taxi_test[features])
mean_squared_error(taxi_test[y_col], preds, squared=False)

<hr>

## Next Steps

In this tutorial, you learned how to use `xgboost` to train a tree-based supervised learning model on a single machine.

Next, try [this xgboost + Dask notebook](./xgboost-dask.ipynb) to learn how to use Dask to train larger models and / or reduce training time.

<hr>